In [ ]:
!pip install graphframes==0.8.2

ERROR: Could not find a version that satisfies the requirement graphframes==0.8.2 (from versions: 0.6)
ERROR: No matching distribution found for graphframes==0.8.2


In [14]:
# Import required libraries
from pyspark.sql import SparkSession
from graphframes import GraphFrame
from pyspark.sql.functions import desc, col, lit


spark = SparkSession.builder \
    .appName("Airline Routes Graph") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.0-s_2.12") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")


df = spark.read.csv("airline_routes.csv", header=True, inferSchema=True)


print("=== Original DataFrame ===")
df.show()


vertices = df.select("source_airport").withColumnRenamed("source_airport", "id").distinct()


edges = df.withColumnRenamed("source_airport", "src") \
          .withColumnRenamed("destination_airport", "dst")


print("=== Vertices DataFrame ===")
vertices.show()


print("=== Edges DataFrame ===")
edges.show()


graph = GraphFrame(vertices, edges)


print("Number of Vertices:", graph.vertices.count())
print("Number of Edges:", graph.edges.count())


grouped_edges = graph.edges.groupBy("src", "dst").count() \
    .filter(col("count") > 5) \
    .orderBy(desc("count")) \
    .withColumn("source_color", lit("#3358FF")) \
    .withColumn("destination_color", lit("#FF3F33"))


print("=== Grouped Edges with Count > 5 ===")
grouped_edges.show()


grouped_edges.write.mode("overwrite").option("header", True).csv("output/grouped_airline_routes")

=== Original DataFrame ===
+-------+----------+--------------+-----------------+-------------------+----------------------+---------+------+---------+
|airline|airline ID|source_airport|source_airport_id|destination_airport|destination_airport_id|codeshare| stops|equipment|
+-------+----------+--------------+-----------------+-------------------+----------------------+---------+------+---------+
|     5T|      1623|           YRT|              132|                YEK|                    50|     NULL|     1|      ATR|
|     AC|       330|           ABJ|              253|                BRU|                   302|     NULL|     1|      333|
|     AC|       330|           YVR|              156|                YBL|                    30|     NULL|     1|      BEH|
|     CU|      1936|           FCO|             1555|                HAV|                  1909|     NULL|     1|      767|
|     FL|      1316|           HOU|             3566|                SAT|                  3621|     NULL